In [91]:
import pandas as pd 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
from pymongo import MongoClient
import pickle

client = MongoClient('mongodb://localhost:27017/')
db = client['movie-web']
col_movies = db['movies']
cur_movies = col_movies.find()
col_actormovies = db['actormovies']
cur_actormovies = col_actormovies.find()
col_actors = db['actors']
cur_actors = col_actors.find()
list_movies = []
list_actormovies = []
list_actors = []
for i in cur_movies:
    list_movies.append(i)
for i in cur_actormovies:
    list_actormovies.append(i)
for i in cur_actors:
    list_actors.append(i)
movies = pd.DataFrame(list_movies)
actormovies = pd.DataFrame(list_actormovies)
actors = pd.DataFrame(list_actors)
movies = movies.drop(['img', 'imgTitle', 'createdAt', 'updatedAt', 'trailer', 'year', 'limit', 'isSeries', '__v', 'numRate', 'rate', 'duration', 'countView', 'epNum'], axis=1)

df = pd.merge(actormovies[['movie', 'actor', 'character']],actors[['_id','name']],left_on='actor', right_on='_id', how='left').drop(columns = ['_id'])
df2 = pd.merge(movies, df, left_on='_id', right_on='movie', how='left').drop(columns = ['movie', 'actor'])
df2 = df2.fillna('')
df2 = df2.groupby('_id').agg({'title':'first', 
                              'desc':'first',
                              'genre':'first',
                             'character': ' '.join, 
                             'name': ' '.join,  }).reset_index()

df2['tags'] = df2['title'] + ' ' + df2['desc'] + ' ' + df2['genre'] + ' ' + df2['character'] + ' ' + df2['name']

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df2['tags'])

with open('vectorizer.pk', 'wb') as fin:
    pickle.dump(tfidf, fin)
    pickle.dump(tfidf_matrix, fin)
    pickle.dump(df2['_id'], fin)

In [92]:
# def get_recommendations(title):

#     df3 = pd.concat([df2['tags'], pd.Series([title])], ignore_index = True)
    
#     tfidf = TfidfVectorizer(stop_words='english')

#     tfidf_matrix = tfidf.fit_transform(df3)
    
#     cosine_sim = linear_kernel(tfidf_matrix[df3.index.max()], tfidf_matrix)
    
#     sim_scores = list(enumerate(cosine_sim[0]))
#     sim_scores = sim_scores[:-1]
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     sim_scores = sim_scores[0:10]

#     movie_indices = [i[0] for i in sim_scores]

#     return df2['title'].iloc[movie_indices].astype('string')

# list(get_recommendations('hoạt hình'))
from sklearn.metrics.pairwise import linear_kernel
import pickle
from sklearn.decomposition import TruncatedSVD
from scipy import sparse

with open('vectorizer.pk', 'rb') as fin:
    vectorizer = pickle.load(fin)
    matrix_tfidf = pickle.load(fin)
    df = pickle.load(fin)

def get_recommendations(title):
    m_new = sparse.vstack((matrix_tfidf, vectorizer.transform([title])))
    svd = TruncatedSVD(n_components=10)
    svd.fit(m_new)
    svd_tfidf_vector = svd.transform(m_new)
    svd_query = np.reshape(svd_tfidf_vector[-1],(1,svd_tfidf_vector[-1].size))
    cosine_sim = linear_kernel(svd_query, svd_tfidf_vector)
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sim_scores[:-1]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:10]
    movie_indices = [i[0] for i in sim_scores]
    return df2['title'].iloc[movie_indices].astype('string')
list(get_recommendations('Phù thủy Stranger'))

['Doctor Strange in Multiverse of Madness',
 'Stranger Things 4',
 'Coco',
 'Nope',
 'Inside Out',
 'Death on the Nile']

In [3]:
userratings = db['userratings']
cursor = userratings.find()
list_cur = []
for i in cursor:
    list_cur.append(i)
ratings = pd.DataFrame(list_cur)
ratings = ratings.drop(['_id', '__v', 'createdAt', 'updatedAt'], axis=1)
ratings

,movie,user,rating
0,633dc3b4f611a788116b43d8,633e8ba2bc3a40391db45a5c,5
1,633dc3b4f611a788116b43d8,633e900bbc3a40391db45a7f,5
2,633dc52cf611a788116b43dc,633b30e4edfee7840d1006c6,4
3,633dc52cf611a788116b43dc,633e8ba2bc3a40391db45a5c,5
4,633dd3e4f611a788116b43e7,633e8ba2bc3a40391db45a5c,2
5,633dd3e4f611a788116b43e7,633b30e4edfee7840d1006c6,2
6,633dc52cf611a788116b43dc,633e900bbc3a40391db45a7f,3
7,6354ebb6c834f95698ef02ca,633e8ba2bc3a40391db45a5c,2
8,6354ec56c834f95698ef02cd,633e8ba2bc3a40391db45a5c,5


In [9]:
def get_items_rated_by_user(user_id):
    y = ratings.iloc[:,1].astype('string')
    ids = np.where(y == user_id)[0]
    item_ids = ratings.iloc[ids, 0].astype('string')
    scores = ratings.iloc[ids, 2]
    return (item_ids, scores)
get_items_rated_by_user('633e8ba2bc3a40391db45a5c')

(0    633dc3b4f611a788116b43d8
 3    633dc52cf611a788116b43dc
 4    633dd3e4f611a788116b43e7
 7    6354ebb6c834f95698ef02ca
 8    6354ec56c834f95698ef02cd
 Name: movie, dtype: string,
 0    5
 3    5
 4    2
 7    2
 8    5
 Name: rating, dtype: int64)

Int64Index([1, 6], dtype='int64')

In [40]:
from sklearn.linear_model import Ridge
from sklearn import linear_model
svd = TruncatedSVD(n_components=10)
svd.fit(matrix_tfidf)
svd_tfidf_vector = svd.transform(matrix_tfidf)
d = svd_tfidf_vector.shape[1] 
users = ratings['user'].astype('string').unique()
W = np.zeros((d, users.size))
b = np.zeros((1, users.size))
# df = pd.DataFrame(df)
for idx, n in enumerate(users):    
    ids, scores = get_items_rated_by_user(n)
    clf = Ridge(alpha = 7, fit_intercept  = True)
    ids_items = df[df.astype('string').isin(ids)].index.tolist()
    Xhat = svd_tfidf_vector[ids_items, :]
    clf.fit(Xhat, scores) 
    W[:, idx] = clf.coef_
    b[0, idx] = clf.intercept_
print(W)
Yhat = svd_tfidf_vector.dot(W) + b
Yhat

[[-5.69028128e-02  1.49714474e-17 -6.83897680e-17]
 [ 1.13697486e-01  9.49623069e-02 -3.25474886e-17]
 [-2.25338663e-01  2.45122529e-16 -1.25000000e-01]
 [ 1.49661337e-01 -1.25000000e-01  1.25000000e-01]
 [-9.73211042e-02 -8.12844405e-02 -3.44521284e-17]
 [-2.45276530e-01 -1.09760854e-16 -5.45810272e-17]]


array([[3.95237064, 4.125     , 3.        ],
       [3.95237064, 3.875     , 3.125     ],
       [3.57737064, 4.        , 2.875     ],
       [3.82579958, 4.01928544, 3.        ],
       [3.60218765, 4.        , 3.        ],
       [3.91570042, 4.        , 3.        ]])

In [107]:
def recommend(user_id):
    items = ratings['movie'].astype('string').unique()
    a = np.zeros((df.size,))
    recommended_items = []
    items_rated_by_user, score = get_items_rated_by_user(user_id)
    for idx, n in enumerate(df.astype('string')): 
        if n not in items_rated_by_user.tolist():
            a[idx] = Yhat[idx, np.where(users == user_id)[0][0]]
    recommended_items = np.argsort(a)[-10:][::-1]
    return df2['title'].iloc[recommended_items]
recommend('633e900bbc3a40391db45a7f')

3                          Stranger Things 4
5                                 Inside Out
4                                       Coco
2    Doctor Strange in Multiverse of Madness
1                          Death on the Nile
0                                       Nope
Name: title, dtype: object

In [106]:
items = ratings['movie'].astype('string').unique()
a = np.zeros((df.size,))
recommended_items = []
items_rated_by_user, score = get_items_rated_by_user('633e900bbc3a40391db45a7f')
for idx, n in enumerate(df.astype('string')): 
    if n not in items_rated_by_user.tolist():
        a[idx] = Yhat[idx, np.where(users == '633e900bbc3a40391db45a7f')[0][0]]
recommended_items = np.argsort(a)[-10:][::-1]
print(recommended_items)
df2['title'].iloc[recommended_items]

[3 5 4 2 1 0]


3                          Stranger Things 4
5                                 Inside Out
4                                       Coco
2    Doctor Strange in Multiverse of Madness
1                          Death on the Nile
0                                       Nope
Name: title, dtype: object